In [1]:
from wykop import WykopAPI
import networkx as nx
import matplotlib
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import time

matplotlib.use('TkAgg')
api = WykopAPI("w55988974f3d3194b7dd98c7ab2c6765c2", "05f093e009943e9e9b911f2a8a9f1a00")
api.authenticate()
tag_name = "polityka"
related = api.make_request(f"tags/{tag_name}/related")
related["data"]

2024-11-18 18:27:07.352 | DEBUG    | wykop.wykop_api:authenticate:42 - Authenticated successfully.
2024-11-18 18:27:07.355 | DEBUG    | wykop.wykop_api:make_request:64 - Making request to https://wykop.pl/api/v3/tags/polityka/related


[{'name': '4konserwy'},
 {'name': 'polska'},
 {'name': 'heheszki'},
 {'name': 'neuropa'},
 {'name': 'usa'},
 {'name': 'konfederacja'},
 {'name': 'pis'},
 {'name': 'wydarzenia'},
 {'name': 'ciekawostki'},
 {'name': 'gospodarka'},
 {'name': 'bekazkonfederacji'},
 {'name': 'trump'},
 {'name': 'ekonomia'},
 {'name': 'europa'},
 {'name': 'swiat'}]

In [2]:
G = nx.DiGraph()
stack = [ w(tag_name, 0)]
max_depth = 4
visited = set()  # Zbiór do śledzenia odwiedzonych tagów
max_retries = 5  # Maksymalna liczba prób dla żądania API

while stack:
    current_tag, depth = stack.pop()

    if depth >= max_depth or current_tag in visited:
        continue

    visited.add(current_tag)  # Oznaczamy tag jako odwiedzony

    retries = 0
    related_data = None

    # Obsługa żądań z ponowieniem w przypadku błędu
    while retries < max_retries:
        try:
            related_data = api.make_request(f"tags/{current_tag}/related")
            break  # Wyjście z pętli, jeśli żądanie się powiodło
        except Exception as e:
            retries += 1
            print(f"Błąd podczas pobierania danych dla '{current_tag}': {e}. Próba {retries}/{max_retries}.")
            time.sleep(2)  # Krótka pauza przed ponownym żądaniem

    if related_data is None:
        print(f"Nie udało się pobrać danych dla '{current_tag}' po {max_retries} próbach.")
        continue

    time.sleep(0.5)

    for related_tag in related_data["data"]:
        G.add_edge(current_tag, related_tag["name"])
        stack.append((related_tag["name"], depth + 1))

NameError: name 'w' is not defined

In [8]:
for related_tag in related["data"]:
    time.sleep(0.5)
    # Dodajemy krawędź skierowaną od głównego tagu do related_tag
    G.add_edge(tag_name, related_tag["name"], weight=1)  
    
    second_degree_related = api.make_request(f"tags/{related_tag['name']}/related")
    for second_degree_tag in second_degree_related["data"]:
        # Dodajemy krawędź skierowaną od related_tag do second_degree_tag
        G.add_edge(related_tag["name"], second_degree_tag["name"], weight=1)  
        third_degree_related = api.make_request(f"tags/{second_degree_tag['name']}/related")
        for third_degree_tag in third_degree_related["data"]:
            time.sleep(0.01)
            # Dodajemy krawędź skierowaną od second_degree_tag do third_degree_tag
            G.add_edge(second_degree_tag["name"], third_degree_tag["name"], weight=1)  
            fourth_degree_related = api.make_request(f"tags/{third_degree_tag['name']}/related")
            for fourth_degree_tag in fourth_degree_related["data"]:
                time.sleep(0.01)
                # Dodajemy krawędź skierowaną od third_degree_tag do fourth_degree_tag
                G.add_edge(third_degree_tag["name"], fourth_degree_tag["name"], weight=1)
                fifth_degree_related = api.make_request(f"tags/{fourth_degree_tag['name']}/related")
                for fifth_degree_tag in fifth_degree_related["data"]:
                    G.add_edge(fourth_degree_tag["name"], fifth_degree_tag["name"], weight=1)
                    sixth_degree_related = api.make_request(f"tags/{fifth_degree_tag['name']}/related")
                    time.sleep(0.01)
                    for sixth_degree_tag in sixth_degree_related["data"]:
                        G.add_edge(fifth_degree_tag["name"], sixth_degree_tag["name"], weight=1)
                        time.sleep(0.001)
                        seventh_degree_related = api.make_request(f"tags/{sixth_degree_tag['name']}/related")
                        for seventh_degree_tag in seventh_degree_related["data"]:
                            G.add_edge(sixth_degree_tag["name"], seventh_degree_tag["name"], weight=1)
                            time.sleep(0.001)

2024-11-18 18:53:12.495 | DEBUG    | wykop.wykop_api:make_request:64 - Making request to https://wykop.pl/api/v3/tags/4konserwy/related
2024-11-18 18:53:12.585 | DEBUG    | wykop.wykop_api:make_request:64 - Making request to https://wykop.pl/api/v3/tags/neuropa/related
2024-11-18 18:53:12.636 | DEBUG    | wykop.wykop_api:make_request:64 - Making request to https://wykop.pl/api/v3/tags/4konserwy/related
2024-11-18 18:53:12.684 | DEBUG    | wykop.wykop_api:make_request:64 - Making request to https://wykop.pl/api/v3/tags/neuropa/related
2024-11-18 18:53:12.706 | DEBUG    | wykop.wykop_api:make_request:64 - Making request to https://wykop.pl/api/v3/tags/4konserwy/related
2024-11-18 18:53:12.760 | DEBUG    | wykop.wykop_api:make_request:64 - Making request to https://wykop.pl/api/v3/tags/neuropa/related
2024-11-18 18:53:13.023 | DEBUG    | wykop.wykop_api:make_request:64 - Making request to https://wykop.pl/api/v3/tags/konfederacja/related
2024-11-18 18:53:13.285 | DEBUG    | wykop.wykop_ap

KeyboardInterrupt: 

In [9]:
print(G) 

DiGraph with 1658 nodes and 5698 edges


In [10]:
plt.figure(figsize=(150, 100))
pos = nx.spring_layout(G, k=0.5, iterations=50)

# nodes
nx.draw_networkx_nodes(G, pos, node_size=[150+ v * 40 for v in dict(G.degree).values()])

# edges
nx.draw_networkx_edges(G, pos, edgelist=G.edges(), width=1.5)

# labels
nx.draw_networkx_labels(G, pos, font_size=2, font_family='sans-serif', font_color='red')

plt.show()

In [12]:
print(G)
nx.write_gml(G, "graph.gml")

# Odczyt grafu z pliku GML
G_loaded = nx.read_gml("graph.gml")
print(G_loaded)

DiGraph with 1658 nodes and 5698 edges
DiGraph with 1658 nodes and 5698 edges
